Import packages

In [30]:
import numpy as np
import pandas as pd
from cube.api import Cube     
cube=Cube(verbose=True)
import re
import nltk
from nltk.corpus import stopwords
from load_baseline_crf import *

In [31]:
df = pd.read_csv("./Jobs_NYC_Postings_20231126.csv")
df = df.loc[:]
df = df.dropna(subset=['Minimum Qual Requirements', 'Preferred Skills'], how='all')

Stopwords Removal

In [32]:
target = df.loc[:, ['Job ID', 'Minimum Qual Requirements', 'Preferred Skills']]
target = target.fillna('nan')

In [33]:
dic = ['', '¢', 'â', '*']
def remove_special_character(text, dic):
    s = ''
    for word in text:
        if word not in dic:
            s += word
    return s

In [34]:
target['Minimum Qual Requirements'] = target['Minimum Qual Requirements'].apply(lambda x : remove_special_character(x, dic))
target['Preferred Skills'] = target['Preferred Skills'].apply(lambda x : remove_special_character(x, dic))
target['skills'] = target['Minimum Qual Requirements'] + target['Preferred Skills']
target = target.drop(columns = ['Minimum Qual Requirements', 'Preferred Skills'])
target

,Job ID,skills
0,606346,1. A masters degree from an accredited colleg...
1,571361,At least six years of full-time satisfactory e...
2,600579,1. A baccalaureate degree from an accredited c...
3,569683,1. A baccalaureate degree from an accredited c...
4,614260,A four-year high school diploma or its educati...
...,...,...
6181,614585,"nanCompletion of FEMA IS-100, 200, 700, and 80..."
6182,616195,"1. For Assignment Level I (only physical, bio..."
6183,611205,"1. For Assignment Level I (only physical, bio..."
6184,602272,1. A masters degree from an accredited colleg...


Sentence Segmentation + boi

In [35]:
cube.load("en", device='cpu')

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
def boi(txt):
    try:
        txt = cube(txt)
        sentence_num = 1
        lemma, pos, sentence_number, tag = [], [], [], []
        for sentence in txt.sentences:
            for token in sentence.tokens:
                for word in token.words:
                    pos.append(word.xpos)
                    lemma.append(word.lemma)
                    sentence_number.append(sentence_num)
                    tag.append('?')
            sentence_num += 1
    except:
        sentence_num = 1
        lemma, pos, sentence_number, tag = [], [], [], []
        for s in nltk.sent_tokenize(txt):
            pos_tagged_sentences = nltk.pos_tag(nltk.word_tokenize(s))
            for item in pos_tagged_sentences:
                pos.append(item[1])
                lemma.append(item[0])
                sentence_number.append(sentence_num)
                tag.append('?')
        sentence_num += 1
                
    df_pos = pd.DataFrame({'sentence_id': sentence_number,
                           'word': lemma,
                           'pos': pos,
                           'tag': tag})

    X_test = prepare_prompt(df_pos)
    y_pred = crf.predict(X_test)
        
    boi_list = []
    for l in y_pred:
        boi_list += l
    
    df_pos['tag'] = boi_list
    # only keeps the columns where tag not equals to O
    
    b, s, result = False, '', []
    for row in df_pos.iterrows():
        if row[1]['tag'][0] == 'B':
            b = True
            if row[1]['word'] != '.<UNK>' and row[1]['word'] != '<UNK>':
                s += str(row[1]['word'])
                s += ' '
        elif b and row[1]['tag'][0] == 'I':
            if row[1]['word'] != '.<UNK>' and row[1]['word'] != '<UNK>':
                s += str(row[1]['word'])
                s += ' '
        else:
            if b:
                s = s[: -1]
                result.append(s)
            b = False
            s = ''

    return result

In [37]:
target['Extracted_Skills'] = target['skills'].apply(lambda x : boi(x))

In [38]:
target.to_csv('result_quanliang.csv')